In [1]:
# series temporales: poder predictivo poco
# hacer streamlit: visualization potente 

# ideas: 
# feature engineering, crear campos: año de convocatoria, diff entre 1a lista y nota final, porcentaje de bajada, ciudad, comunidad, 
# numero de listas o convocatorias que han habido por universidad, diferencia de notas entre años

# limpieza: quitar columnas de sobre 10, concatenar todas las particiones de años, nombres de universidades, columnas, asegurar fechas correctas, 
# todas las notas con el mismo numero de decimales

# modelo: predecir la nota (a través de un rango, con una tolerancia que defina) para el año 2022-2023 estudiando los datos de años anteriores. 
# Predecir también la probabilidad de que entres en una universidad u otra en funcion de la nota que hayas sacado en una convocatoria anterior
# adicionalmente, tener en cuenta que la metodologia del examen cambió por el COVID

# visualizacion: mapa geografico de españa, con un mapa de calor por la probabilidad o por la nota mas alta y mas baja
# visualizacion de datos historicos a traves de un grafico de linea
# crecimiento con los años
# front end que se pueda filtrar por universidad, convocatoria o nota final, 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

In [3]:
current_directory = os.getcwd()
raw_data_directory = current_directory + '/raw_data'
raw_data_directory

'/Users/miguel/repos/Prediction_Medicine_Selectivity_Scores/raw_data'

# Importing data

In [4]:
# input raw data from different years (2010-2021)
initial_year = 2010
last_year = 2021
next_year = initial_year + 1
num_years = last_year - initial_year + 1
list_of_dfs = []
list_of_directories = []
# format example: df_2010_2011 = pd.read_csv(raw_data_directory + '/Notas de corte de Medicina 2010_2011 - Hoja 1.csv', decimal = ',')

for i in range(num_years):
    df_name = 'df' + '_' + str(initial_year) + '_' + str(next_year)
    directory = raw_data_directory + '/Notas de corte de Medicina ' + str(initial_year) + '_' + str(next_year) + ' - Hoja 1.csv'
    list_of_dfs.append(df_name)
    list_of_directories.append(directory)
    
    list_of_dfs[i] = pd.read_csv(list_of_directories[i], decimal = ',')
    
    initial_year += 1
    next_year += 1

In [5]:
# before appending all dfs, we create a column with the year, to keep track of which df the data is coming from
initial_year = 2010

for i in range(num_years):
    list_of_dfs[i]['year'] = initial_year + i

In [6]:
dfs_unified = pd.DataFrame()

In [7]:
for i in range(num_years):
    dfs_unified = dfs_unified.append(list_of_dfs[i])

In [8]:
# check the append has been done correctly: check the number of rows is the sum of all of the dfs

total_number_of_rows = 0

for i in range(num_years):
    total_number_of_rows = total_number_of_rows + len(list_of_dfs[i])

if total_number_of_rows != dfs_unified.shape[0]:
    sys.exit()
else:
    print('append has been done correctly')

append has been done correctly


# Data Cleaning

## Check for duplicates

In [9]:
if dfs_unified[dfs_unified.duplicated() == True].shape[0] != 0:
    dfs_unified.drop_duplicates()
else:
    print('There are no duplicates')

There are no duplicates


## Create new unique columns out of data that we already had

In [10]:
def drop_columns_containing_text(df, text):
    '''
    drop columns that contain a certain string/text
    '''
    df = df[df.columns.drop(list(df.filter(regex=text)))]
    return df

In [11]:
dfs_unified = drop_columns_containing_text(dfs_unified, 'Sobre')

In [12]:
# to keep track of raw columns, to afterwards drop them
dfs_unified_raw_columns = dfs_unified.columns.tolist()
# 'year' was created before and is not a raw column
dfs_unified_raw_columns.remove('year')
#dfs_unified_columns

In [13]:
# tiene sentido copiar solo not null data y no toda????????

In [14]:
def copy_data(new_col, old_col):
    '''
    copy not null data from one column to another
    '''
    dfs_unified[new_col] = np.where(dfs_unified[old_col].notnull(), dfs_unified[old_col], dfs_unified[new_col])

In [15]:
dfs_unified['university'] = ''
copy_data('university', 'Notas de corte de Medicina')
copy_data('university', 'Universidad')
copy_data('university', 'Facultad de Medicina')

dfs_unified['date_last_admission'] = ''
copy_data('date_last_admission', 'Fecha de última admisión')
copy_data('date_last_admission', 'Última fecha de admisión \n(Fecha de la nota final)')

dfs_unified['final_grade'] = ''
copy_data('final_grade', 'Nota de corte final de Medicina')
copy_data('final_grade', 'Nota de corte final')
copy_data('final_grade', 'Nota de corte actual')


for i in range(1, 24):
    dfs_unified[str(i) + '_list'] = ''
    copy_data(str(i) + '_list', str(i) + 'ª Lista')

copy_data('19_list', '19ª lista')
copy_data('20_list', '20ª lista')

In [16]:
# drop all old columns
dfs_unified = dfs_unified.drop(dfs_unified_raw_columns, axis=1)

# reset index
dfs_unified = dfs_unified.reset_index()

In [17]:
dfs_unified[~dfs_unified['university'].str.contains('U', na=False)]

,index,year,university,date_last_admission,final_grade,1_list,2_list,3_list,4_list,5_list,...,14_list,15_list,16_list,17_list,18_list,19_list,20_list,21_list,22_list,23_list
31,31,2010,,,,,,,,,...,,,,,,,,,,
33,33,2010,"** Publicada nota de 11,257 en la CiuG, el 7/9/10","Y RECTIFICADA POR 11,410; 8/9/10",,,,,,,...,,,,,,,,,,
98,32,2012,Nota promedio,,12.017,,,,,,...,,,,,,,,,,
99,33,2012,Desviación típica,,0.22,,,,,,...,,,,,,,,,,
132,32,2013,Nota promedio,,12.082,,,,,,...,,,,,,,,,,
133,33,2013,Desviación típica,,0.213,,,,,,...,,,,,,,,,,
166,32,2014,Nota promedio,,12.268,,,,,,...,,,,,,,,,,
167,33,2014,Desviación típica,,0.213,,,,,,...,,,,,,,,,,
200,32,2015,Nota promedio,,12.422,12.605,12.517,12.48,12.455,,...,,,,,,,,,,
201,33,2015,Desviación típica,,0.188,0.201,0.177,0.175,0.173,,...,,,,,,,,,,


In [18]:
dfs_unified = dfs_unified[dfs_unified['university'].str.contains('U', na=False)]

In [19]:
dfs_unified = dfs_unified[~dfs_unified['university'].str.contains('Sobre la Universidad de Santiago:', na=False)]

In [20]:
dfs_unified.columns

Index(['index', 'year', 'university', 'date_last_admission', 'final_grade',
       '1_list', '2_list', '3_list', '4_list', '5_list', '6_list', '7_list',
       '8_list', '9_list', '10_list', '11_list', '12_list', '13_list',
       '14_list', '15_list', '16_list', '17_list', '18_list', '19_list',
       '20_list', '21_list', '22_list', '23_list'],
      dtype='object')

In [21]:
dfs_unified_columns = dfs_unified.columns.tolist()[1:]

num_columns = len(dfs_unified_columns)

for i in range(num_columns):
    dfs_unified[dfs_unified_columns[i]].replace({'': np.nan}, inplace=True)

In [22]:
# quality check: essential columns don't have nulls

nulls_date_last_admission = dfs_unified[dfs_unified['date_last_admission'].isnull()].shape[0]
nulls_1_list = dfs_unified[dfs_unified['1_list'].isnull()].shape[0]
nulls_final_grade = dfs_unified[dfs_unified['final_grade'].isnull()].shape[0]
nulls_university = dfs_unified[dfs_unified['university'].isnull()].shape[0]

if nulls_date_last_admission or nulls_1_list or nulls_final_grade or nulls_university != 0:
    sys.exit()
else:
    print('column date_last_admission has', nulls_date_last_admission, 'null rows')
    print('column 1_list has', nulls_1_list, 'null rows')
    print('column final_grade has', nulls_final_grade, 'null rows')
    print('column university has', nulls_university, 'null rows')

column date_last_admission has 0 null rows
column 1_list has 0 null rows
column final_grade has 0 null rows
column university has 0 null rows


## Create consistent University names across years

In [23]:
dfs_unified['university'] = dfs_unified['university'].str.lower()

In [24]:
last_year = 2021
dfs_unified[dfs_unified['year'] == last_year]['university'].values

array(['universidad complutense de madrid', 'universidad de sevilla',
       'universidad de murcia', 'universidad de granada',
       'u. autónoma de madrid', 'universidad de málaga',
       'universidad de córdoba', 'universidad de cádiz',
       'u. miguel hernández (s. juan de alicante)',
       'u. de c-la mancha (campus c.real)', 'universidad de alcalá',
       'universidad de valladolid', 'urjc - campus alcorcón',
       'u. de c-la mancha (campus albacete)',
       'u. de extremadura (badajoz)', 'universidad pública de navarra',
       'universidad de valencia', 'universidad jaume i',
       'u. de zaragoza (campus zaragoza)',
       'universidad de barcelona-clínico', 'universidad de la laguna',
       'universidad de las palmas', 'universidad de salamanca',
       'u. de zaragoza (campus huesca)', 'u. de santiago de compostela',
       'universidad de oviedo', 'u. de cantabria (santander)',
       'universidad de barcelona-bellvitge',
       'universidad del país vasco (lejon

In [25]:
list_of_uni_names_2021 = dfs_unified[dfs_unified['year'] == last_year]['university'].tolist()
num_of_uni_names_2021 = len(list_of_uni_names_2021)
print('There are', num_of_uni_names_2021, 'universities in 2021')

dfs_unified_list_uni = list(dfs_unified['university'].unique())
num_of_uni_names_dataset = len(dfs_unified_list_uni)
print('There are a total of', num_of_uni_names_dataset, 'universities in our dataset')

diff_uni_names = num_of_uni_names_dataset - num_of_uni_names_2021
print('This means there are a total of', diff_uni_names, 'universities that have a different name than the ones in our 2021 data, and need to be mapped')

There are 35 universities in 2021
There are a total of 48 universities in our dataset
This means there are a total of 13 universities that have a different name than the ones in our 2021 data, and need to be mapped


In [26]:
# we create a list with the universities that have different name

list_not_matched = []

for i in dfs_unified_list_uni:
    if i not in list_of_uni_names_2021:
        list_not_matched.append(i)
    else:
        continue


if len(list_not_matched) != diff_uni_names:
    sys.exit()
else:
    print('list of not matched created successfully')

list of not matched created successfully


### Sequence Matcher

In [27]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [28]:
df1 = pd.DataFrame({
    "not_matched_uni": list_not_matched
})

df2 = pd.DataFrame({
    "list_of_uni_names_2021": list_of_uni_names_2021
})

df3 = pd.MultiIndex.from_product(
    [df1["not_matched_uni"], df2["list_of_uni_names_2021"]], names=["not_matched_uni", "list_of_uni_names_2021"]
).to_frame(index=False)

In [29]:
df3['ratio'] = ''

for i in range(len(df3['not_matched_uni'])):
    df3['ratio'][i] = similar(df3['not_matched_uni'][i], df3['list_of_uni_names_2021'][i])

df3['not_matched_uni'].nunique()

13

In [30]:
df4 = df3.groupby('not_matched_uni')['ratio'].max().reset_index()

df5 = pd.merge(df4,df3,on='not_matched_uni',how='left')
df5 = df5[df5['ratio_x'] == df5['ratio_y']]
df5

,not_matched_uni,ratio_x,list_of_uni_names_2021,ratio_y
31,universidad autónoma de madrid,0.825397,universidad autónoma de barcelona,0.825397
54,universidad de barcelona,0.857143,universidad de barcelona-clínico,0.857143
96,universidad de cantabria (santander),0.825397,u. de cantabria (santander),0.825397
134,universidad de clm- campus de albacete,0.676056,universidad de las islas baleares,0.676056
140,universidad de clm- campus de ciudad real,0.648649,universidad complutense de madrid,0.648649
189,universidad de extremadura (badajoz),0.825397,u. de extremadura (badajoz),0.825397
227,universidad de jaume i,0.926829,universidad jaume i,0.926829
269,universidad de santiago de compostela,0.830769,u. de santiago de compostela,0.830769
303,universidad de zaragoza - campus de huesca,0.750000,u. de zaragoza (campus huesca),0.75
333,universidad de zaragoza - campus de zaragoza,0.763158,u. de zaragoza (campus zaragoza),0.763158


In [31]:
# hacerlo con str.contains, crear funcion

In [32]:
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de cantabria (santander)'), 'u. de cantabria (santander)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de extremadura (badajoz)'), 'u. de extremadura (badajoz)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de jaume i'), 'universidad jaume i', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de zaragoza - campus de huesca'), 'u. de zaragoza (campus huesca)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de santiago de compostela'), 'u. de santiago de compostela', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de zaragoza - campus de zaragoza'), 'u. de zaragoza (campus zaragoza)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad miguel hernández (san juan de alicante)'), 'u. miguel hernández (s. juan de alicante)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad pompeu fabra (barcelona)'), 'u. pompeu fabra (barcelona)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de santiago de compostela'), 'u. de santiago de compostela', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad rey juan carlos - campus de alcorcón'), 'urjc - campus alcorcón', dfs_unified['university'])


dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad autónoma de madrid'), 'urjc - campus alcorcón', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de barcelona'), 'urjc - campus alcorcón', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de clm- campus de albacete'), 'urjc - campus alcorcón', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de clm- campus de ciudad real'), 'urjc - campus alcorcón', dfs_unified['university'])

In [33]:
# quality check: make sure all university names are mapped with 2021 names

dfs_unified_list_uni = list(dfs_unified['university'].unique())

list_not_matched = []

for i in dfs_unified_list_uni:
    if i not in list_of_uni_names_2021:
        list_not_matched.append(i)
    else:
        continue

# checking length of not matched
if len(list_not_matched) != 0:
    sys.exit()
else:
    print('all university names are consistent with 2021')

# another way of checking this could be by analyzing the overall length and unique names
if dfs_unified['university'].nunique() != len(list_of_uni_names_2021):
    sys.exit()
else:
    print('all university names are consistent with 2021')

all university names are consistent with 2021
all university names are consistent with 2021


## Convert columns with numbers to floats

In [34]:
# to convert to float, we can't have commas or spaces, as this function '.astype(float)' will give back an error

In [35]:
# float to 2 decimals?? some have 3

In [36]:
for i in range(1,24):
    try:
        dfs_unified[str(i) + '_list'] = dfs_unified[str(i) + '_list'].astype(float)
    except:
        dfs_unified[str(i) + '_list'] = np.where((dfs_unified[str(i) + '_list'].str.contains(',', na=False)), dfs_unified[str(i) + '_list'].replace({',': '.'}, inplace=True), dfs_unified[str(i) + '_list'])
        dfs_unified[str(i) + '_list'] = np.where((dfs_unified[str(i) + '_list'] == ''), dfs_unified[str(i) + '_list'].replace({'': np.nan}, inplace=True), dfs_unified[str(i) + '_list'])
        dfs_unified[str(i) + '_list'] = np.where((dfs_unified[str(i) + '_list'] == 's/datos'), dfs_unified[str(i) + '_list'].replace({'s/datos': np.nan}, inplace=True), dfs_unified[str(i) + '_list'])

In [37]:
# check date_last_admission format

In [38]:
dfs_unified[dfs_unified['year'] == 2021]

,index,year,university,date_last_admission,final_grade,1_list,2_list,3_list,4_list,5_list,...,14_list,15_list,16_list,17_list,18_list,19_list,20_list,21_list,22_list,23_list
383,0,2021,universidad complutense de madrid,29/09/2021,13.426,13.500,13.455,13.446,13.440,13.428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,1,2021,universidad de sevilla,24/09/2021,13.345,13.530,13.445,13.400,13.390,13.376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,2,2021,universidad de murcia,17/09/2021,13.345,13.558,13.492,13.449,13.419,13.345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,3,2021,universidad de granada,24/09/2021,13.319,13.580,13.494,13.450,13.410,13.386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,4,2021,u. autónoma de madrid,28/10/2021,13.309,13.463,13.407,13.350,13.340,13.332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,5,2021,universidad de málaga,09/09/2021,13.275,13.476,13.414,13.360,13.338,13.320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,6,2021,universidad de córdoba,24/09/2021,13.250,13.450,13.395,13.335,13.317,13.310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,7,2021,universidad de cádiz,24/09/2021,13.232,13.430,13.38,13.314,13.294,13.280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,8,2021,u. miguel hernández (s. juan de alicante),06/10/2021,13.150,13.419,13.34,13.270,13.250,13.240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,9,2021,u. de c-la mancha (campus c.real),27/10/2021,13.146,13.421,13.339,13.280,13.260,13.146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
jj

NameError: name 'jj' is not defined

In [ ]:
# Export data to be used on Tableau

In [ ]:
file_name = 'output/exported_data.csv'
dfs_unified.to_csv(file_name, index = False, header=True)

# Data Visualization

In [ ]:
complutense = dfs_unified[dfs_unified['university'] == 'universidad complutense de madrid']
fig,ax=plt.subplots(figsize=(10,8))
ax.bar(complutense['year'], complutense['1_list'])
# Setting the x-axis to 1-10
# and y-axis to 1-15
plt.axis([2009, 2022, 11.00, 14.00])
plt.show()

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st

In [ ]:
st.title('My title')

In [ ]:
# end data visualization

# Feature Engineering

In [ ]:
# creat city column - PASAR A MAS ABAJO

In [ ]:
# first: bigger cities in case two names appear
list_of_cities = ['madrid', 'zaragoza', 'barcelona', 'sevilla', 'valencia', 'murcia', 'granada','málaga', 'córdoba', 'cádiz', 'alicante', 'albacete', \
    'valladolid', 'badajoz', 'gerona', 'lérida', 'reus', 'navarra', 'santander', 'oviedo', \
    'santiago de compostela', 'salamanca', 'país vasco', 'salamanca', 'navarra', 'las palmas', 'la laguna', 'alcalá', 'ciudad real', 'huesca']

In [ ]:
num_cities = len(list_of_cities)

In [ ]:
i=0
dfs_unified['city'] = ''
while i < num_cities:
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains(list_of_cities[i], na=False), list_of_cities[i], dfs_unified['city'])
    i +=1

In [ ]:
dfs_unified['city'].replace({'': np.nan}, inplace=True)

In [ ]:
dfs_unified[dfs_unified['city'].isnull()]

In [ ]:
dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('alcorcón', na=False), 'madrid', dfs_unified['city'])
dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('c.real', na=False), 'ciudad real', dfs_unified['city'])
dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('jaume', na=False), 'castellon de la plana', dfs_unified['city'])
dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('islas baleares', na=False), 'palma de mallorca', dfs_unified['city'])

In [ ]:
dfs_unified[dfs_unified['city'].isnull()]

In [ ]:
num_columns = len(dfs_unified_columns)

while i < num_columns:
    dfs_unified[dfs_unified_columns[i]].replace({'': np.nan}, inplace=True)
    i += 1

In [ ]:
dfs_unified['place'] = np.where(dfs_unified['university'].str.contains('universidad de', na=False), dfs_unified['university'].str.strip().str[15:], dfs_unified['place'])
dfs_unified['place'] = np.where(dfs_unified['university'].str.contains('u. de', na=False), dfs_unified['university'].str.strip().str[6:], dfs_unified['place'])

In [ ]:
dfs_unified['university'].str.split(' ')

In [ ]:
#dfs_unified[dfs_unified['year'] == 2021]

In [ ]:
u_de = dfs_unified[dfs_unified['university'].str.contains('u. de', na=False)]
dfs_unified['place'] = u_de['university'].str.strip().str[6:]

In [ ]:
dfs_unified[~dfs_unified['university'].str.contains('Un', na=False)]

In [ ]:
dfs_unified['place']

In [ ]:
dfs_unified['city']

In [ ]:
dfs_unified['campus']

In [ ]:
dfs_unified['campus']

In [ ]:
# new university column
# university + city + campus

In [ ]:
# nombres de universidades
# duplicados
# notas de mayor a menor

In [ ]:
dfs_unified[dfs_unified['university'] == 'Sobre la Universidad de Santiago:']

In [ ]:
# año-univeristy y ver cuales no aparecen en el año anterior
dfs_unified['year_university'] = (dfs_unified['year']).astype(str) + '_' + dfs_unified['university']

In [ ]:
uni_year_grouped = dfs_unified.groupby(['year'])['university'].count()
uni_year_grouped

# Model

In [ ]:
# CREATE COLUMN THAT IS THE DIFFERENCE BETWEEN YEARS, AND PUT IT IN THE MODEL AS X, together with year

In [ ]:
# Regression: Supervised ML, we have a variable target, and want to predict a quantity

In [40]:
dfs_unified_columns = dfs_unified.columns.tolist()[1:]

num_columns = len(dfs_unified_columns)

for i in range(num_columns):
    dfs_unified[dfs_unified_columns[i]].replace({np.nan: 0}, inplace=True)

In [ ]:
dfs_unified_columns

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
jj

In [ ]:
#next_year = 2022
#df_2022_2023 = pd.DataFrame()
#df_2022_2023['university'] = pd.Series(dfs_unified[dfs_unified['year'] == last_year]['university'])
#df_2022_2023['year'] = next_year

In [ ]:
#dfs_unified = dfs_unified.append(df_2022_2023)
#dfs_unified = dfs_unified.reset_index()
#dfs_unified = dfs_unified.drop(['level_0', 'index'], axis=1)
dfs_unified

In [ ]:
# hacer un filtrado por univeristy y crear dfs por una unica universidad, de tal manera que  X = year (varia entre 201 y 2021), e y = 1_list

In [45]:

universidad_de_lerida = dfs_unified[dfs_unified['university'] == 'universidad de lérida']
universidad_de_lerida = universidad_de_lerida[['year', '1_list']]

# change in test format due to covid
universidad_de_lerida['covid_format_change'] = ''
universidad_de_lerida['covid_format_change'] = np.where((universidad_de_lerida['year'] >= 2020), float(1), universidad_de_lerida['covid_format_change'])
universidad_de_lerida['covid_format_change'] = np.where((universidad_de_lerida['year'] < 2020), float(0), universidad_de_lerida['covid_format_change'])

# growth column
universidad_de_lerida['growth'] = ''
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2010), float(0), universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2011), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2011, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2010, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2012), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2012, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2011, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2013), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2013, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2012, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2014), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2014, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2013, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2015), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2015, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2014, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2016), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2016, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2015, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2017), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2017, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2016, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2018), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2018, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2017, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2019), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2019, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2018, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2020), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2020, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2019, '1_list'].iloc[0])) , universidad_de_lerida['growth'])
universidad_de_lerida['growth'] = np.where((universidad_de_lerida['year'] == 2021), float((universidad_de_lerida.loc[universidad_de_lerida['year'] == 2021, '1_list'].iloc[0]) - (universidad_de_lerida.loc[universidad_de_lerida['year'] == 2020, '1_list'].iloc[0])) , universidad_de_lerida['growth'])

In [46]:
universidad_de_lerida

,year,1_list,covid_format_change,growth
0,2010,11.654,0.0,0.0
35,2011,11.650,0.0,-0.004
68,2012,11.944,0.0,0.294
100,2013,12.000,0.0,0.056
134,2014,12.044,0.0,0.044
168,2015,12.266,0.0,0.222
204,2016,12.305,0.0,0.039
240,2017,12.311,0.0,0.006
275,2018,12.264,0.0,-0.047
309,2019,12.293,0.0,0.029


In [47]:
universidad_de_lerida['growth'].average()

AttributeError: 'Series' object has no attribute 'average'

In [ ]:
universidad_de_lerida.loc[universidad_de_lerida['year'] == 2011, '1_list'].iloc[0]

In [ ]:
# let's check the correlation between the two variables
sns.heatmap(universidad_de_lerida.corr(), cmap='YlGnBu', annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(universidad_de_lerida['year'], universidad_de_lerida['1_list'], color='red')
plt.title('Year vs 1_list', fontsize = 14)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('1_list', fontsize = 14)
plt.grid(True)
plt.show()

In [43]:
# Create a Linear regression

lr = LinearRegression()
X = universidad_de_lerida[['year', 'covid_format_change']]
y = universidad_de_lerida[['1_list']]

lr.fit(X, y)

LinearRegression()

In [44]:
# predict 1_list for a given year
lr.predict([[2022, 1.0]])


array([[12.85823494]])

In [ ]:
# generate model prediction for all years in dataset
y_predict = lr.predict(X)

# visualize the predicted score as a line on the test set
plt.figure(figsize=(12,6))
plt.scatter(universidad_de_lerida['year'], universidad_de_lerida['1_list'], color='red')
plt.plot(universidad_de_lerida['year'], y_predict, color='g')
plt.xlabel('Year', fontsize = 14)
plt.ylabel('1_list', fontsize = 14)
plt.show()

In [ ]:
# compare model predictions vs actual values
plt.figure(figsize=(12,6))
x_ax = range(len(universidad_de_lerida['1_list']))
plt.plot(x_ax, universidad_de_lerida['1_list'], label='original')
plt.plot(x_ax, y_predict, label='predicted')
plt.title('Actual 1_list scores vs predicted')
plt.legend()
plt.show()

In [ ]:
# NO TENGO QUE HACER TRAIN Y TEST

lr = LinearRegression()
X = test_df[['1_list']]
y = test_df[['1_list']]
# X, y
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
print(y_hat.shape)


In [ ]:
test_df

In [ ]:
test_df = dfs_unified[dfs_unified['year'] == last_year]

In [ ]:
dfs_unified[['1_list']]

In [ ]:
y_hat

In [ ]:
lr.predict()

## Metrics for regression

MSE: Mean Squared Error

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - h(x_i))^2$$

MAE: Mean Absolute Error 

$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - h(x_i)|$$

MAPE: Mean Absolute Percent Error

$$MAE = \frac{1}{n} \sum_{i=1}^{n} \frac{|y_i - h(x_i)|}{y_i}$$

Explained Variance:


$$explained\_{}variance(y, \hat{y}) = 1 - \frac{Var\{ y - \hat{y}\}}{Var\{y\}}$$



We will learn more about scoring and model selection in a later module

In [ ]:
errors = y - y_predict
mae = np.mean(np.abs(errors))
mse = np.mean(errors ** 2)
rmse = np.sqrt(mse)

In [ ]:
results_models = pd.DataFrame(columns = ['MAE', 'MSE', 'RMSE'])
results_models.loc['linear_regression'] = [mae, mse, rmse]
results_models